In [1]:
import arcpy
from scripts._6m_caltrans_activities import CalTrans
from sys import argv
from scripts.utils import init_gdb, check_exists, og_file_input
import os
import datetime
original_gdb, workspace, scratch_workspace = init_gdb()
arcpy.env.qualifiedFieldNames=False

# INPUTS
# will need to download updated versions and update file names
# veg_pt = og_file_input(prefix = 'Vegetation_Control_', filetype = 'Point', gdb = os.path.join(workspace, "a_Originals"))
# input_pts = os.path.join(workspace, "a_Originals", veg_pt)

# veg_poly = og_file_input(prefix = 'Vegetation_Control_', filetype = 'Polyline', gdb = os.path.join(workspace, "a_Originals"))
# input_polys = os.path.join(workspace, "a_Originals", veg_poly)
input_polys21 = 'C:/Users/sageg/Documents/ArcGIS/Projects/PC414 CWI Million Acres/1-Spatial Data/CalTrans/Caltrans_Vegetation_Management_Program_20230614.gdb/VMP_Highway_Summary_2021'
input_polys22 = 'C:/Users/sageg/Documents/ArcGIS/Projects/PC414 CWI Million Acres/1-Spatial Data/CalTrans/Caltrans_Vegetation_Management_Program_20230614.gdb/VMP_Highway_Summary_2022'

input_table21 = 'C:/Users/sageg/Documents/ArcGIS/Projects/PC414 CWI Million Acres/1-Spatial Data/CalTrans/Caltrans_Vegetation_Management_Program_20230614.gdb/VMP_Activities_Table_2021'
input_table22 = 'C:/Users/sageg/Documents/ArcGIS/Projects/PC414 CWI Million Acres/1-Spatial Data/CalTrans/Caltrans_Vegetation_Management_Program_20230614.gdb/VMP_Activities_Table_2022'

# OUTPUTS
date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216
output_lines_standardized = os.path.join(workspace, "c_Standardized", f'CalTrans_act_ln_standardized_{date_id}')
# output_points_standardized = os.path.join(workspace, "c_standardized", f'CalTrans_act_pts_standardized_{date_id}')
# output_points_enriched = os.path.join(workspace, "d_Enriched",f'CalTrans_act_pts_enriched_{date_id}') 
output_lines_enriched = os.path.join(workspace, "d_Enriched",f'CalTrans_act_ln_enriched_{date_id}') 


In [2]:
# CalTrans(input_polys21, input_polys22, input_table21, input_table22, output_lines_standardized, output_lines_enriched)

In [3]:
import arcpy
from scripts._1b_add_fields import AddFields
from scripts._2b_assign_domains import AssignDomains
from scripts._7b_enrichments_pts import enrich_points
from scripts._7c_enrichments_lines import enrich_lines
from scripts._2k_keep_fields import KeepFields
import os
from sys import argv
from scripts.utils import init_gdb, delete_scratch_files, runner
import time

original_gdb, workspace, scratch_workspace = init_gdb()

# 6m CalTrans_Activities 20221123
# def CalTrans(input_pts, input_polys, output_lines_standardized, output_points_standardized, output_points_enriched, output_lines_enriched):  
#def CalTrans(input_polys21, input_polys22, input_table21, input_table22, output_lines_standardized, output_lines_enriched):  
start = time.time()
print(f'Start Time {time.ctime()}')
arcpy.env.overwriteOutput = True

arcpy.ImportToolbox(r"c:\program files\arcgis\pro\Resources\ArcToolbox\toolboxes\Data Management Tools.tbx")

# define intermediary objects in scratch
CalTrans21_scratch = os.path.join(scratch_workspace,'CalTrans21_scratch')
CalTrans22_scratch = os.path.join(scratch_workspace,'CalTrans22_scratch')


Start Time Tue Jul 11 14:41:01 2023


In [6]:
### BEGIN POLYLINE WORKFLOW
# Process: Add Join (2) (Add Join) (management)
print("     step 3/33 add join")
input_table21_join = arcpy.management.AddJoin(
                                in_layer_or_view=input_polys21, 
                                in_field="HIghwayID", 
                                join_table=input_table21, 
                                join_field="HighwayID", 
                                join_type="KEEP_ALL", 
                                index_join_fields="INDEX_JOIN_FIELDS"
                                )

# Process: Copy Features (Copy Features) (management)
input_table21_join_copy = arcpy.management.CopyFeatures(
        input_table21_join, 
        CalTrans21_scratch
        )

input_table22_join = arcpy.management.AddJoin(
                                in_layer_or_view=input_polys22, 
                                in_field="HIghwayID", 
                                join_table=input_table22, 
                                join_field="Highway_ID", 
                                join_type="KEEP_ALL", 
                                index_join_fields="INDEX_JOIN_FIELDS"
                                )

# Process: Copy Features (2) (Copy Features) (management)
input_table22_join_copy = arcpy.management.CopyFeatures(
        input_table22_join, 
        CalTrans22_scratch
        )

# alter1 = arcpy.management.AlterField(CalTrans22_scratch, 'Highway_ID', 'HighwayID')

     step 3/33 add join


In [11]:

print('Appending Lines')
# Create Featureclass (Data Management) to append into
create1 = arcpy.management.CreateFeatureclass(scratch_workspace, "CalTransLns_scratch", "POLYLINE", 
                                    CalTrans22_scratch)

# Append (Data Management) Caltrans Lines
# append1 = arcpy.management.Append([CalTrans22_scratch, CalTrans21_scratch], 
#                         create1, "NO_TEST")

CalTransLns_scratch_3_ = arcpy.management.Append(inputs=[CalTrans22_scratch, 
                                                         CalTrans21_scratch], 
                                                         target=create1, 
                                                         schema_type="NO_TEST", 
                                                         field_mapping=""""DISTRICT_CODE \"DISTRICT_CODE\" true true false 255 Text 0 0,First,#,
                                                         CalTrans22_scratch,DISTRICT_CODE,0,255,CalTrans21_scratch,DISTRICT_CODE,0,255;
                                                         CO \"COUNTY_CODE\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,CO,0,255,CalTrans21_scratch,CO,0,255;
                                                         RT \"ROUTE_NAME\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,RT,0,255,CalTrans21_scratch,RT,0,255;
                                                         ROUTE_SUFFIX_CODE \"ROUTE_SUFFIX_CODE\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,ROUTE_SUFFIX_CODE,0,255,CalTrans21_scratch,ROUTE_SUFFIX_CODE,0,255;
                                                         ROUTE_NUM \"ROUTE\" true true false 2 Short 0 0,First,#,CalTrans22_scratch,ROUTE_NUM,-1,-1,CalTrans21_scratch,ROUTE_NUM,-1,-1;
                                                         ROUTE_C \"ROUTE_C\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,ROUTE_C,0,255,CalTrans21_scratch,ROUTE_C,0,255;
                                                         HIghwayID \"Highway ID\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,HIghwayID,0,255,CalTrans21_scratch,HIghwayID,0,255;
                                                         FREQUENCY \"FREQUENCY\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,FREQUENCY,-1,-1,CalTrans21_scratch,FREQUENCY,-1,-1;
                                                         SUM_Production_Quantity \"SUM_Production_Quantity\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,SUM_Production_Quantity,-1,-1,CalTrans21_scratch,SUM_Production_Quantity,-1,-1;
                                                         OBJECTID_1 \"OBJECTID\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,OBJECTID_1,-1,-1,CalTrans21_scratch,OBJECTID_1,-1,-1;
                                                         GISID \"GISID\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,GISID,-1,-1,CalTrans21_scratch,GISID,-1,-1;
                                                         Resp__District \"Resp__District\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Resp__District,-1,-1;
                                                         Resp__Region \"Resp__Region\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Resp__Region,-1,-1;
                                                         Resp__Area \"Resp__Area\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Resp__Area,-1,-1;
                                                         Resp_ \"Resp_\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Resp_,-1,-1;
                                                         Work_Order_Number \"Work_Order_Number\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Work_Order_Number,-1,-1,CalTrans21_scratch,Work_Order_Number,-1,-1;
                                                         County \"County\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,County,0,8000,CalTrans21_scratch,County,0,8000;
                                                         TwoCounties \"TwoCounties\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,TwoCounties,0,8000,CalTrans21_scratch,TwoCounties,0,255;
                                                         Route \"Route\" true true false 4 Text 0 0,First,#,CalTrans22_scratch,Route,0,4,CalTrans21_scratch,Route,0,4;
                                                         From_PM \"From_PM\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,From_PM,0,8000,CalTrans21_scratch,From_PM,0,8000;
                                                         To_PM \"To_PM\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,To_PM,0,8000,CalTrans21_scratch,To_PM,0,8000;
                                                         Point_or_Line \"Point_or_Line\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Point_or_Line,0,8000,CalTrans21_scratch,Point_or_Line,0,255;
                                                         Production_Quantity \"Production_Quantity\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,Production_Quantity,-1,-1,CalTrans21_scratch,Production_Quantity,-1,-1;
                                                         UOM \"UOM\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,UOM,0,8000,CalTrans21_scratch,UOM,0,8000;
                                                         Maintenance_Type \"Maintenance_Type\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Maintenance_Type,0,8000,CalTrans21_scratch,Maintenance_Type,0,8000;
                                                         IMMS_Project_Code \"IMMS_Project_Code\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,IMMS_Project_Code,0,8000,CalTrans21_scratch,IMMS_Project_Code,0,8000;
                                                         EFIS_Reporting_Code \"EFIS_Reporting_Code\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,EFIS_Reporting_Code,0,8000,CalTrans21_scratch,EFIS_Reporting_Code,0,8000;
                                                         EFIS_Project_Code \"EFIS_Project_Code\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,EFIS_Project_Code,-1,-1,CalTrans21_scratch,EFIS_Project_Code,-1,-1;
                                                         Comments \"Comments\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Comments,0,8000,CalTrans21_scratch,Comments,0,8000;
                                                         Work_Order_Priority \"Work_Order_Priority\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Work_Order_Priority,0,8000,CalTrans21_scratch,Work_Order_Priority,0,8000;
                                                         Activity \"Activity\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Activity,0,8000,CalTrans21_scratch,Activity,0,8000;
                                                         Activity_Description \"Activity_Description\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Activity_Description,0,8000,CalTrans21_scratch,Activity_Description,0,8000;
                                                         IMMS_Unit_ID \"IMMS_Unit_ID\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,IMMS_Unit_ID,0,8000,CalTrans21_scratch,IMMS_Unit_ID,0,8000;
                                                         EFIS_Sobj \"EFIS_Sobj\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,EFIS_Sobj,-1,-1,CalTrans21_scratch,EFIS_Sobj,-1,-1;
                                                         Prob \"Prob\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Prob,0,8000,CalTrans21_scratch,Prob,0,8000;
                                                         Fiscal_Year \"Fiscal_Year\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Fiscal_Year,-1,-1,CalTrans21_scratch,Fiscal_Year,-1,-1;
                                                         Labor_Hours \"Labor_Hours\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,Labor_Hours,-1,-1,CalTrans21_scratch,Labor_Hours,-1,-1;
                                                         P_Y_s \"P_Y_s\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,P_Y_s,-1,-1,CalTrans21_scratch,PYs,-1,-1;
                                                         Labor_Cost \"Labor_Cost\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,Labor_Cost,-1,-1,CalTrans21_scratch,Labor_Cost,-1,-1;
                                                         Vehicle_Cost \"Vehicle_Cost\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,Vehicle_Cost,-1,-1,CalTrans21_scratch,Vehicle_Cost,-1,-1;
                                                         Material_Cost \"Material_Cost\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Material_Cost,0,8000,CalTrans21_scratch,Material_Cost,-1,-1;
                                                         Other_Cost \"Other_Cost\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Other_Cost,0,8000,CalTrans21_scratch,Other_Cost,-1,-1;
                                                         Total_Cost \"Total_Cost\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Total_Cost,0,8000,CalTrans21_scratch,Total_Cost,-1,-1;
                                                         From_Miles \"From_Miles\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,From_Miles,-1,-1,CalTrans21_scratch,From_Miles,-1,-1;
                                                         To_Miles \"To_Miles\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,To_Miles,-1,-1,CalTrans21_scratch,To_Miles,-1,-1;
                                                         Secondary_Prod \"Secondary_Prod\" true true false 8 Double 0 0,First,#,CalTrans22_scratch,Secondary_Prod,-1,-1,CalTrans21_scratch,Secondary_Prod,-1,-1;
                                                         Secondary_UOM \"Secondary_UOM\" true true false 8000 Text 0 0,First,#,CalTrans22_scratch,Secondary_UOM,0,8000,CalTrans21_scratch,Secondary_UOM,0,8000;
                                                         Route_Numeric \"Route_Numeric\" true true false 4 Long 0 0,First,#,CalTrans22_scratch,Route_Numeric,-1,-1,CalTrans21_scratch,Route_Numeric,-1,-1;
                                                         Work_Order_Text \"Work_Order_Text\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,Work_Order_Text,0,255;
                                                         District_Text \"District_Text\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,District_Text,0,255,CalTrans21_scratch,District_Text,0,255;
                                                         GISID_Text \"GISID_Text\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,GISID_Text,0,255,CalTrans21_scratch,GISID_Text,0,255;
                                                         Highway_ID \"Highway_ID\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,Highway_ID,0,255,CalTrans21_scratch,HIghwayID,0,255;
                                                         Route_Suffix \"Route_Suffix\" true true false 255 Text 0 0,First,#,CalTrans22_scratch,Route_Suffix,0,255,CalTrans21_scratch,Route_Suffix,0,255"""
                                                         )



Appending Lines


In [12]:

print('Performing Line Standardization')
# Process: Copy Features (2) (Copy Features) (management)
# caltrans_poly_copy = os.path.join(scratch_workspace, "CalTrans_pts_Copy")
# arcpy.management.CopyFeatures(in_features=append1, 
#                                 out_feature_class=caltrans_poly_copy, 
#                                 config_keyword="", 
#                                 spatial_grid_1=None, 
#                                 spatial_grid_2=None, 
#                                 spatial_grid_3=None)

# Process: Repair Geometry (Repair Geometry) (management)
caltrans_poly_copy_repaired_geom = arcpy.management.RepairGeometry(in_features=append1, 
                                                        delete_null="KEEP_NULL", 
                                                        validation_method="ESRI")

# Process: Alter Field County (2) (Alter Field) (management)
caltrans_poly_alterfield_v1 = arcpy.management.AlterField(in_table=caltrans_poly_copy_repaired_geom, 
                                                        field="County", 
                                                        new_field_name="County2", 
                                                        new_field_alias="County2", 
                                                        field_type="TEXT", 
                                                        #field_length=25, 
                                                        #field_is_nullable="NULLABLE", 
                                                        clear_field_alias="DO_NOT_CLEAR")

# Process: Alter Field Activity Description (2) (Alter Field) (management)
caltrans_poly_alterfield_v2 = arcpy.management.AlterField(in_table=caltrans_poly_alterfield_v1, 
                                                        field="Activity_Description", 
                                                        new_field_name="Activity_Description_", 
                                                        new_field_alias="", 
                                                        field_type="TEXT", 
                                                        #field_length=70, 
                                                        #field_is_nullable="NULLABLE", 
                                                        clear_field_alias="DO_NOT_CLEAR")

# Process: 1b Add Fields (2) (1b Add Fields) (PC414CWIMillionAcres)
caltrans_poly_addfields = AddFields(Input_Table=caltrans_poly_alterfield_v2)

# Process: Calculate Project ID (2) (Calculate Field) (management)
caltrans_poly_calc_field_v1 = arcpy.management.CalculateField(in_table=caltrans_poly_addfields, 
                                                            field="PROJECTID_USER", 
                                                            expression="!HighwayID!", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Agency (2) (Calculate Field) (management)
caltrans_poly_calc_field_v2 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v1, 
                                                            field="AGENCY", 
                                                            expression="\"CALSTA\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Data Steward (2) (Calculate Field) (management)
caltrans_poly_calc_field_v3 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v2, 
                                                            field="ORG_ADMIN_p", 
                                                            expression="\"CALTRANS\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Data Steward (3) (Calculate Field) (management)
caltrans_poly_calc_field_v4 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v3, 
                                                            field="ORG_ADMIN_a", 
                                                            expression="\"CALTRANS\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Data Steward (4) (Calculate Field) (management)
caltrans_poly_calc_field_v5 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v4, 
                                                            field="ORG_ADMIN_a", 
                                                            expression="\"CALTRANS\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Project Contact (2) (Calculate Field) (management)
caltrans_poly_calc_field_v6 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v5, 
                                                            field="PROJECT_CONTACT", 
                                                            expression="\"Division of Maintenance\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Email (2) (Calculate Field) (management)
caltrans_poly_calc_field_v7 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v6, 
                                                            field="PROJECT_EMAIL", 
                                                            expression="\"andrew.lozano@dot.ca.gov\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Admin Org (2) (Calculate Field) (management)
caltrans_poly_calc_field_v8 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v7, 
                                                        field="ADMINISTERING_ORG", 
                                                        expression="CALTRANS", 
                                                        expression_type="PYTHON3", 
                                                        code_block="", 
                                                        field_type="TEXT", 
                                                        enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Primary Funding Source (2) (Calculate Field) (management)
caltrans_poly_calc_field_v9 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v8, 
                                                            field="PRIMARY_FUNDING_SOURCE", 
                                                            expression="\"GENERAL_FUND\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Primary Funding Org (2) (Calculate Field) (management)
caltrans_poly_calc_field_v10 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v9, 
                                                            field="PRIMARY_FUNDING_ORG", 
                                                            expression="\"CALTRANS\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Treatment ID (2) (Calculate Field) (management)
Updated_Input_Table_46_ = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v10, 
                                                          field="TRMTID_USER", 
                                                          expression="!HIghwayID!+'-'+!From_PM!+'-'+!To_PM!", 
                                                          expression_type="PYTHON3", 
                                                          code_block="", 
                                                          field_type="TEXT", 
                                                          enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate WUI (2) (Calculate Field) (management)
caltrans_poly_calc_field_v11 = arcpy.management.CalculateField(in_table=Updated_Input_Table_46_, 
                                                            field="IN_WUI", 
                                                            expression="ifelse(!WUI!)", 
                                                            expression_type="PYTHON3", 
                                                            code_block="""def ifelse(WUI):
                                                                if WUI == \"Yes\":
                                                                    return \"WUI_USER_DEFINED\"
                                                                elif WUI == \"No\":
                                                                    return \"NON-WUI_USER_DEFINED\"
                                                                else:
                                                                    return WUI""", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Treatment Area (2) (Calculate Field) (management)
caltrans_poly_calc_field_v12 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v11, 
                                                            field="TREATMENT_AREA", 
                                                            expression="ifelse(!UOM!, !Production_Quantity!)", 
                                                            expression_type="PYTHON3", 
                                                            code_block="""def ifelse(UOM, Q):
                                                                if UOM == \"ACRE\":
                                                                    return Q
                                                                else:
                                                                    return None""", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Activity ID (2) (Calculate Field) (management)
caltrans_poly_calc_field_v13 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v12, 
                                                                    field="ACTIVID_USER", 
                                                                    expression="!Work_Order_Number!", 
                                                                    expression_type="PYTHON3", 
                                                                    code_block="", 
                                                                    field_type="TEXT", 
                                                                    enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Implementing Org (2) (Calculate Field) (management)
caltrans_poly_calc_field_v14 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v13, 
                                                            field="IMPLEMENTING_ORG", 
                                                            expression="!District!", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Activity UOM (2) (Calculate Field) (management)
caltrans_poly_calc_field_v15 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v14, 
                                                            field="ACTIVITY_UOM", 
                                                            expression="ifelse(!UOM!)", 
                                                            expression_type="PYTHON3", 
                                                            code_block="""def ifelse(Unit):
                                                                        if Unit == 'ACRE':
                                                                            return 'AC'
                                                                        else:
                                                                            return Unit""", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Activity Quantity (2) (Calculate Field) (management)
caltrans_poly_calc_field_v16 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v15, 
                                                            field="ACTIVITY_QUANTITY", 
                                                            expression="!Production_Quantity!", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Status (2) (Calculate Field) (management)
caltrans_poly_calc_field_v17 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v16, 
                                                            field="ACTIVITY_STATUS", 
                                                            expression="COMPLETE", 
                                                            expression_type="PYTHON3", 
                                                            code_block="""
                                                                        """, 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Activity Start (2) (Calculate Field) (management)
caltrans_poly_calc_field_v18 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v17, 
                                                            field="ACTIVITY_START", 
                                                            expression="None", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")
                                                            
# Process: Calculate Activity End (2) (Calculate Field) (management)
caltrans_poly_calc_field_v19 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v18, 
                                                            field="ACTIVITY_END", 
                                                            expression="'8/1/'+str(!Fiscal_Year!)", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Source (2) (Calculate Field) (management)
caltrans_poly_calc_field_v20 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v19, 
                                                            field="Source", 
                                                            expression="\"CALTRANS\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Crosswalk (2) (Calculate Field) (management)
caltrans_poly_calc_field_v21 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v20, 
                                                            field="Crosswalk", 
                                                            expression="!Activity_Description_!", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Geometry (Calculate Field) (management)
caltrans_poly_calc_field_v22 = arcpy.management.CalculateField(in_table=caltrans_poly_calc_field_v21, 
                                                            field="TRMT_GEOM", 
                                                            expression="'LINE'", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: Calculate Geometry Attributes (2) (Calculate Geometry Attributes) (management)
# CalTrans_pts_Copy_15_ = arcpy.management.CalculateGeometryAttributes(in_features=caltrans_poly_calc_field_v19, 
#                                                                      geometry_property=[["LATITUDE", "INSIDE_Y"], ["LONGITUDE", "INSIDE_X"]], 
#                                                                      length_unit="", 
#                                                                      area_unit="", 
#                                                                      coordinate_system="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]", 
#                                                                      coordinate_format="DD")

# Process: Delete Field (2) (Delete Field) (management)
caltrans_poly_keepfields = KeepFields(caltrans_poly_calc_field_v21)

print(f'Saving Output Lines Standardized: {output_lines_standardized}')
# Process: Copy Features (3) (Copy Features) (management)
arcpy.management.CopyFeatures(in_features=caltrans_poly_keepfields, 
                                out_feature_class=output_lines_standardized, 
                                config_keyword="", 
                                spatial_grid_1=None, 
                                spatial_grid_2=None, 
                                spatial_grid_3=None)

# Process: 2b Assign Domains (3) (2b Assign Domains) (PC414CWIMillionAcres)
caltrans_line_standardized_assigndomains = AssignDomains(in_table=output_lines_standardized)

print('Performing Lines Enrichments')
# Process: 7c Enrichments Lines (2) (7c Enrichments Lines) (PC414CWIMillionAcres)
caltrans_lines_enriched = enrich_lines(line_fc=caltrans_line_standardized_assigndomains) # don't delete scratch

print(f'Saving Output Lines Enriched: {output_lines_enriched}')
# Process: Copy Features (4) (Copy Features) (management)
arcpy.management.CopyFeatures(in_features=caltrans_lines_enriched, 
                                out_feature_class=output_lines_enriched, 
                                config_keyword="", 
                                spatial_grid_1=None, 
                                spatial_grid_2=None, 
                                spatial_grid_3=None)

# Process: Calculate Owner State (2) (Calculate Field) (management)
caltrans_lines_enriched_calc_field_v1 = arcpy.management.CalculateField(in_table=output_lines_enriched, 
                                                            field="PRIMARY_OWNERSHIP_GROUP", 
                                                            expression="\"STATE\"", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

## *** IMMS_Unit_ID and IMMS_ID no longer exist after running a 7 tool on the data, so changed to 'TREATMENT_ID_USER' since that was filled with IMMS 
## *** prior to the 7 tool being ran. Get non-specific error (no line traceback) that nontypes and string types cannot be concatenated when 
## *** running the script with this block of code intact. The output table "caltrans_lines_enriched_calc_field_v2" should be fed into AssignDomains below once functioning.
# Process: Calculate Treatment ID (2) (Calculate Field) (management)
caltrans_lines_enriched_calc_field_v2 = arcpy.management.CalculateField(in_table=caltrans_lines_enriched_calc_field_v1, 
                                                            field="TREATMENT_ID_USER", 
                                                            expression="!PROJECTID_USER!+'-'+!COUNTY![:8]+'-'+!REGION![:3]+'-'+!IN_WUI![:3]", 
                                                            expression_type="PYTHON3", 
                                                            code_block="", 
                                                            field_type="TEXT", 
                                                            enforce_domains="NO_ENFORCE_DOMAINS")

# Process: 2b Assign Domains (4) (2b Assign Domains) (PC414CWIMillionAcres)
caltrans_lines_enriched_assigndomains = AssignDomains(in_table=caltrans_lines_enriched_calc_field_v2)

print('Deleting Scratch Files')
delete_scratch_files(gdb = scratch_workspace, delete_fc = 'yes', delete_table = 'yes', delete_ds = 'yes')

end = time.time()
print(f'Time Elapsed: {(end-start)/60} minutes')
# if __name__ == '__main__':
#      runner(workspace,scratch_workspace,CalTrans, '*argv[1:]')


Performing Line Standardization


ExecuteError: ERROR 000539: Invalid field District
Failed to execute (CalculateField).
